In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import re, pickle, os, csv, ast
import functools, itertools
import json, string

from string import digits
from openai import OpenAI
from dotenv import load_dotenv
from collections import defaultdict
from deep_translator import GoogleTranslator

In [2]:
# Load API key from .env file
load_dotenv()

key = os.getenv('OPENAI_API_KEY')
if key is None:
    raise ValueError("The OPENAI_API_KEY environment variable is not set \
                     or .env file is missing.")

client = OpenAI(
    api_key=key
)

In [3]:
def call_openai_api(user_prompt, system_prompt, n_runs=1, model="gpt-4-turbo-2024-04-09"):
    responses = []
    for run_number in range(1, n_runs + 1):
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
        )
        response_content = completion.choices[0].message.content
        print(response_content)
        print("========================================next call")
        responses.append(response_content)
    return responses

In [4]:
def load_prompt_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def generate_user_prompt(file_path):
    return load_prompt_from_file(file_path)

def generate_system_prompt(file_path):
    return load_prompt_from_file(file_path)


In [5]:
def normalize_keys(entry):
    # Define a mapping from variant keys to a standard key
    key_map = {
        'cost': 'costs',
        'pay': 'costs',
        'values': 'costs',
        'calorie content': 'calorie',
        'calories': 'calorie',
        'proteins': 'protein',
        'carbs': 'carbohydrates',
        'carbons': 'carbohydrates',
        'counties': 'carbohydrates',
        'carbohydrate': 'carbohydrates',
        'big': 'fat',
        'bold': 'fat'
    }
    # Normalize keys according to the mapping
    return {key_map.get(key, key): value for key, value in entry.items()}


In [6]:
# def parse_to_dataframe(json_strings):
#     all_entries = []
#     # Define the expected set of keys after normalization
#     expected_keys = {'name', 'costs', 'calorie', 'protein', 'fat', 'carbohydrates'}
    
#     for json_str in json_strings:
#         # Convert single quotes to double quotes to make it valid JSON
#         json_str = json_str.replace("\'", "\"")
#         # Find the starting index of the JSON array, assumed to start with '['
#         start_index = json_str.find('[')
#         if start_index != -1:
#             # Slice from the start index to the end of the string
#             json_str = json_str[start_index:]
#             try:
#                 # Parse the JSON string
#                 data = json.loads(json_str)
#                 # Normalize keys and filter the entries
#                 normalized_data = [normalize_keys(entry) for entry in data]
#                 filtered_entries = [entry for entry in normalized_data if set(entry.keys()) == expected_keys]
#                 all_entries.extend(filtered_entries)
#             except json.JSONDecodeError:
#                 print("Error decoding JSON")
    
#     # Convert the list of dictionaries to a DataFrame
#     return pd.DataFrame(all_entries)

# def fix_json_format(json_str):
#     # Replace incorrect single quotes with double quotes
#     json_str = json_str.replace("\'", "\"")
#     # Replace semicolons used incorrectly with commas
#     json_str = json_str.replace(";", ",")
#     # Handle cases where single quotes are used improperly
#     json_str = json_str.replace("‘", "\"").replace("’", "\"")
#     return json_str


# def parse_to_dataframe(json_strings):
#     all_entries = []
#     expected_keys = {'name', 'costs', 'calorie', 'protein', 'fat', 'carbohydrates'}
    
#     for json_str in json_strings:
#         cleaned_json = fix_json_format(json_str)
#         try:
#             data = json.loads(cleaned_json)
#             normalized_data = [normalize_keys(entry) for entry in data]
#             filtered_entries = [entry for entry in normalized_data if set(entry.keys()) == expected_keys]
#             all_entries.extend(filtered_entries)
#         except json.JSONDecodeError as e:
#             print(f"Error decoding JSON: {e}")
    
#     return pd.DataFrame(all_entries)

# def parse_to_dataframe(json_strings):
#     all_entries = []
#     expected_keys = {'name', 'cost', 'calories', 'protein', 'fat', 'carbohydrates'}
    
#     for json_str in json_strings:
#         cleaned_json = fix_json_format(json_str)
#         try:
#             # Ensure it's a proper JSON list
#             if cleaned_json.strip()[0] != '[':
#                 cleaned_json = f"[{cleaned_json}]"
#             data = json.loads(cleaned_json)
#             normalized_data = [normalize_keys(entry) for entry in data]
#             filtered_entries = [entry for entry in normalized_data if set(entry.keys()) == expected_keys]
#             all_entries.extend(filtered_entries)
#         except json.JSONDecodeError as e:
#             print(f"Error decoding JSON: {e} in string {json_str}")
    
#     return pd.DataFrame(all_entries)

def fix_json_format(json_str):
    # Using regular expressions to find the most likely JSON start
    matches = list(re.finditer(r'(\[\{|\{)', json_str))
    if not matches:
        return None

    # Starting from the first match that looks like a JSON start
    start = matches[0].start()
    json_str = json_str[start:]
    
    # Using a stack to handle nested structures properly
    stack = []
    end = start
    for i, char in enumerate(json_str):
        if char in '{[':
            stack.append(char)
        elif char in '}]':
            if not stack:
                break  # Unbalanced bracket
            last = stack.pop()
            if (last == '{' and char != '}') or (last == '[' and char != ']'):
                break  # Mismatched bracket
        if not stack:
            end = i + 1
            break
    
    json_str = json_str[:end].replace("'", '"')  # Ensuring proper quote usage
    return json_str

def parse_to_dataframe(json_strings):
    all_entries = []
    expected_keys = {'name', 'cost', 'calories', 'protein', 'fat', 'carbohydrates'}

    for json_str in json_strings:
        cleaned_json = fix_json_format(json_str)
        if cleaned_json is None:
            print(f"No valid JSON found in string: {json_str[:100]}...")
            continue
        
        try:
            data = json.loads(cleaned_json)
            if isinstance(data, dict):  # Wrap single dictionary in a list
                data = [data]
            if isinstance(data, list):
                filtered_entries = [entry for entry in data if isinstance(entry, dict) and set(entry.keys()) == expected_keys]
                all_entries.extend(filtered_entries)
            else:
                print(f"Unexpected data type: {type(data)} in string: {json_str[:100]}...")
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e} near '{cleaned_json[:100]}'...")

    return pd.DataFrame(all_entries)




In [7]:
def translate_to_eng(responses):
    
    translator = GoogleTranslator(source='auto', target='en')
    translated_texts = []

    for response in responses:
        # Ensure the text does not exceed the 5000 character limit
        if len(response) <= 5000:
            try:
                # Translate the text and convert to lowercase
                translated = translator.translate(response).lower()
                translated_texts.append(translated)
            except Exception as e:
                print(f"Failed to translate text due to: {e}")
                translated_texts.append("Translation failed")
        else:
            print("Text too long to translate:", response)
            translated_texts.append("Text too long and was not translated")

    return translated_texts

### Collect English Version

In [ ]:
user_prompt_path = './prompts/cost_nutri_prompts/user_prompt.txt'
system_prompt_path = './prompts/cost_nutri_prompts/system_prompt.txt'

In [ ]:
user_prompt = generate_user_prompt(user_prompt_path)
system_prompt = generate_system_prompt(system_prompt_path)

In [ ]:
eng_resp = call_openai_api(user_prompt, system_prompt, 30)
eng_resp

In [ ]:
# with open('./responses/cost_nutri/eng_resp.pkl', 'wb') as f:
#     pickle.dump(eng_resp, f)

In [ ]:
with open('./responses/cost_nutri/eng_resp.pkl', 'rb') as f:
    eng_resp = pickle.load(f)

In [ ]:
eng_resp

In [ ]:
eng_df = parse_to_dataframe(eng_resp)

eng_df

In [ ]:
eng_resp2 = call_openai_api(user_prompt, system_prompt, 5)
eng_resp2

In [ ]:
# with open('./responses/cost_nutri/eng_resp2.pkl', 'wb') as f:
#     pickle.dump(eng_resp2, f)

In [ ]:
with open('./responses/cost_nutri/eng_resp2.pkl', 'rb') as f:
    eng_resp2 = pickle.load(f)

In [ ]:
eng_resp2

In [ ]:
eng_df2 = parse_to_dataframe(eng_resp2)

eng_df2

In [ ]:
eng_df_final = pd.concat([eng_df, eng_df2[:30]], ignore_index=True)
eng_df_final

In [ ]:
eng_df_final.to_pickle("../dataset/cost_nutri/eng_result.pkl")

### Collect Chinese Version

In [ ]:
user_prompt_chn = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_chn.txt')
system_prompt_chn = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_chn.txt')

In [ ]:
chn_resp = call_openai_api(user_prompt_chn, system_prompt_chn, 35)
chn_resp

In [ ]:
# with open('./responses/cost_nutri/chn_resp.pkl', 'wb') as f:
#     pickle.dump(chn_resp, f)

In [ ]:
with open('./responses/cost_nutri/chn_resp.pkl', 'rb') as f:
    chn_resp = pickle.load(f)

In [ ]:
chn_resp

In [ ]:
trans_chn_resp = translate_to_eng(chn_resp)
trans_chn_resp

In [ ]:
chn_df = parse_to_dataframe(trans_chn_resp)

chn_df

In [ ]:
chn_df_final = chn_df[:300]
chn_df_final

In [ ]:
chn_df_final.to_pickle("../dataset/cost_nutri/chn_result.pkl")

### Collect Arabic Version

In [ ]:
user_prompt_arab = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_arabic.txt')
system_prompt_arab = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_arabic.txt')

In [ ]:
arab_resp = call_openai_api(user_prompt_arab, system_prompt_arab, 35)
arab_resp

In [ ]:
# with open('./responses/cost_nutri/arab_resp.pkl', 'wb') as f:
#     pickle.dump(arab_resp, f)

In [ ]:
with open('./responses/cost_nutri/arab_resp.pkl', 'rb') as f:
    arab_resp = pickle.load(f)

In [ ]:
trans_arab_resp = translate_to_eng(arab_resp)
trans_arab_resp

In [ ]:
arab_df = parse_to_dataframe(trans_arab_resp)

arab_df

In [ ]:
arab_resp2 = call_openai_api(user_prompt_arab, system_prompt_arab, 35)
arab_resp2

In [ ]:
# with open('./responses/cost_nutri/arab_resp2.pkl', 'wb') as f:
#     pickle.dump(arab_resp2, f)

In [ ]:
with open('./responses/cost_nutri/arab_resp2.pkl', 'rb') as f:
    arab_resp2 = pickle.load(f)

In [ ]:
trans_arab_resp2 = translate_to_eng(arab_resp2)
trans_arab_resp2

In [ ]:
arab_df2 = parse_to_dataframe(trans_arab_resp2)

arab_df2

In [ ]:
arab_df_final = pd.concat([arab_df, arab_df2], ignore_index=True)
arab_df_final

In [ ]:
arab_df_final.to_pickle("../dataset/cost_nutri/arab_result.pkl")

### Collect Filipino Version

In [ ]:
user_prompt_fili = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_filipino.txt')
system_prompt_fili = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_filipino.txt')

In [ ]:
fili_resp = call_openai_api(user_prompt_fili, system_prompt_fili, 35)
fili_resp

In [ ]:
# with open('./responses/cost_nutri/fili_resp.pkl', 'wb') as f:
#     pickle.dump(fili_resp, f)

In [ ]:
with open('./responses/cost_nutri/fili_resp.pkl', 'rb') as f:
    fili_resp = pickle.load(f)

In [ ]:
fili_resp

In [ ]:
trans_fili_resp = translate_to_eng(fili_resp)
trans_fili_resp

In [ ]:
fili_df = parse_to_dataframe(trans_fili_resp)

fili_df

In [ ]:
fili_df_final = fili_df[:300]
fili_df_final

In [ ]:
fili_df_final.to_pickle("../dataset/cost_nutri/fili_result.pkl")

### Collect French Version

In [ ]:
user_prompt_french = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_french.txt')
system_prompt_french = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_french.txt')

In [ ]:
french_resp = call_openai_api(user_prompt_french, system_prompt_french, 35)
french_resp

In [ ]:
with open('./responses/cost_nutri/french_resp.pkl', 'wb') as f:
    pickle.dump(french_resp, f)

In [ ]:
with open('./responses/cost_nutri/french_resp.pkl', 'rb') as f:
    french_resp = pickle.load(f)

In [ ]:
french_resp

In [ ]:
trans_french_resp = translate_to_eng(french_resp)
trans_french_resp

In [ ]:
french_df = parse_to_dataframe(trans_french_resp)

french_df

In [ ]:
french_df_final = french_df[:300]
french_df_final

In [ ]:
french_df_final.to_pickle("../dataset/cost_nutri/french_result.pkl")

### Collect German Version

In [ ]:
user_prompt_german = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_german.txt')
system_prompt_german = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_german.txt')

In [ ]:
german_resp = call_openai_api(user_prompt_german, system_prompt_german, 35)
german_resp

In [ ]:
# with open('./responses/cost_nutri/german_resp.pkl', 'wb') as f:
#     pickle.dump(german_resp, f)

In [ ]:
with open('./responses/cost_nutri/german_resp.pkl', 'rb') as f:
    german_resp = pickle.load(f)

In [ ]:
german_resp

In [ ]:
trans_german_resp = translate_to_eng(german_resp)
trans_german_resp

In [ ]:
german_df = parse_to_dataframe(trans_german_resp)

german_df

In [ ]:
german_df_final = german_df[:300]
german_df_final

In [ ]:
german_df_final.to_pickle("../dataset/cost_nutri/german_result.pkl")

### Collect Greek Version

In [ ]:
user_prompt_greek = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_greek.txt')
system_prompt_greek = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_greek.txt')

In [ ]:
greek_resp = call_openai_api(user_prompt_greek, system_prompt_greek, 35)
greek_resp

In [ ]:
# with open('./responses/cost_nutri/greek_resp.pkl', 'wb') as f:
#     pickle.dump(greek_resp, f)

In [ ]:
with open('./responses/cost_nutri/greek_resp.pkl', 'rb') as f:
    greek_resp = pickle.load(f)

In [ ]:
greek_resp

In [ ]:
trans_greek_resp = translate_to_eng(greek_resp)
trans_greek_resp

In [ ]:
greek_df = parse_to_dataframe(trans_greek_resp)

greek_df

In [ ]:
greek_resp2 = call_openai_api(user_prompt_greek, system_prompt_greek, 10)
greek_resp2

In [ ]:
# with open('./responses/cost_nutri/greek_resp2.pkl', 'wb') as f:
#     pickle.dump(greek_resp2, f)

In [ ]:
with open('./responses/cost_nutri/greek_resp2.pkl', 'rb') as f:
    greek_resp2 = pickle.load(f)

In [ ]:
trans_greek_resp2 = translate_to_eng(greek_resp2)
trans_greek_resp2

In [ ]:
greek_df2 = parse_to_dataframe(trans_greek_resp2)

greek_df2

In [ ]:
greek_df_final = pd.concat([greek_df, greek_df2[:47]], ignore_index=True)
greek_df_final

In [ ]:
greek_df_final.to_pickle("../dataset/cost_nutri/greek_result.pkl")

### Collect Hawaiian Version

In [8]:
user_prompt_hawa = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_hawaiian.txt')
system_prompt_hawa = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_hawaiian.txt')

In [9]:
hawa_resp = call_openai_api(user_prompt_hawa, system_prompt_hawa, 35)
hawa_resp

KeyboardInterrupt: 

In [ ]:
# with open('./responses/cost_nutri/hawa_resp.pkl', 'wb') as f:
#     pickle.dump(hawa_resp, f)

In [10]:
with open('./responses/cost_nutri/hawa_resp.pkl', 'rb') as f:
    hawa_resp = pickle.load(f)

In [11]:
hawa_resp

["Hele mai nei kahi papa inoa o nā meaʻai olakino me kā lākou mau kumukūʻai a me ka ʻikepili meaʻai no kēlā me kēia lawelawe:\n\n1. {'inoa': 'salad ʻōmaʻomaʻo',\n   'uku': 7,\n   'calorie': 150,\n   'polokina': 5,\n   'momona': 10,\n   'nā kalapona': 18}\n\n2. {'inoa': 'pouli kolo',\n   'uku': 8,\n   'calorie': 200,\n   'polokina': 25,\n   'momona': 5,\n   'nā kalapona': 9}\n\n3. {'inoa': 'ʻōhelo māla',\n   'uku': 4,\n   'calorie': 120,\n   'polokina': 3,\n   'momona': 1,\n   'nā kalapona': 25}\n\n4. {'inoa': 'hāʻukeʻuke ʻākoʻakoʻa huakaʻi',\n   'uku': 6,\n   'calorie': 300,\n   'polokina': 8,\n   'momona': 15,\n   'nā kalapona': 35}\n\n5. {'inoa': 'ʻalani salmon',\n   'uku': 13,\n   'calorie': 450,\n   'polokina': 45,\n   'momona': 25,\n   'nā kalapona': 0}\n\n6. {'inoa': 'iʻa pikaula keʻokeʻo',\n   'uku': 11,\n   'calorie': 380,\n   'polokina': 40,\n   'momona': 22,\n   'nā kalapona': 4}\n\n7. {'inoa': 'quinoa a me kāpena',\n   'uku': 9,\n   'calorie': 320,\n   'polokina': 12,\n   'm

In [12]:
trans_hawa_resp = translate_to_eng(hawa_resp)
trans_hawa_resp

["here's a list of healthy foods with their prices and nutritional information for each serving:\n\n1. {'name': 'green salad',\n   'pay': 7,\n   'calories': 150,\n   'protein': 5,\n   'fat': 10,\n   'carbs': 18}\n\n2. {'name': 'dark creep',\n   'pay': 8,\n   'calories': 200,\n   'protein': 25,\n   'fat': 5,\n   'carbons': 9}\n\n3. {'name': 'gardenia',\n   'pay': 4,\n   'calories': 120,\n   'protein': 3,\n   'fat': 1,\n   'carbs': 25}\n\n4. {'name': 'coral tour',\n   'pay': 6,\n   'calories': 300,\n   'protein': 8,\n   'fat': 15,\n   'carbs': 35}\n\n5. {'name': 'orange salmon',\n   'pay': 13,\n   'calories': 450,\n   'protein': 45,\n   'fat': 25,\n   'carbons': 0}\n\n6. {'name': 'white snapper',\n   'pay': 11,\n   'calories': 380,\n   'protein': 40,\n   'fat': 22,\n   'carbons': 4}\n\n7. {'name': 'quinoa and cabbage',\n   'pay': 9,\n   'calories': 320,\n   'protein': 12,\n   'fat': 5,\n   'counties': 55}\n\n8. {'name': 'soft tofu',\n   'pay': 6,\n   'calories': 220,\n   'protein': 20,\n

In [13]:
hawa_df = parse_to_dataframe(trans_hawa_resp)

hawa_df

Error decoding JSON: Expecting ',' delimiter: line 7 column 2 (char 105) near '[{"name": "wood salmon",
  "pay": 15,
  "calories": 233,
  "protein": 25,
  "fat": 13,
  "carbons": '...
Error decoding JSON: Expecting ',' delimiter: line 8 column 2 (char 107) near '[{"name": "pai kale",
  "pay": 3,
  "calories": 50,
  "protein": 3,
  "fat": 1,
  "carbohydrates": 1'...
No valid JSON found in string: 10 healthy foods you should follow:
1. kale salad - kale, other hardy vegetables, almonds, tomatoes,...


""


### Collect Hindi Version

In [ ]:
user_prompt_hindi = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_hindi.txt')
system_prompt_hindi = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_hindi.txt')

In [ ]:
hindi_resp = call_openai_api(user_prompt_hindi, system_prompt_hindi, 35)
hindi_resp

In [ ]:
# with open('./responses/cost_nutri/hindi_resp.pkl', 'wb') as f:
#     pickle.dump(hindi_resp, f)

In [ ]:
with open('./responses/cost_nutri/hindi_resp.pkl', 'rb') as f:
    hindi_resp = pickle.load(f)

In [ ]:
trans_hindi_resp = translate_to_eng(hindi_resp)
trans_hindi_resp

In [ ]:
hindi_df = parse_to_dataframe(trans_hindi_resp)

hindi_df

In [ ]:
hindi_df.to_pickle("../dataset/cost_nutri/hindi_result.pkl")

### Collect Hungarian Version

In [ ]:
user_prompt_hung = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_hungarian.txt')
system_prompt_hung = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_hungarian.txt')

In [ ]:
hung_resp = call_openai_api(user_prompt_hung, system_prompt_hung, 35)
hung_resp

In [ ]:
# with open('./responses/cost_nutri/hung_resp.pkl', 'wb') as f:
#     pickle.dump(hung_resp, f)

In [ ]:
with open('./responses/cost_nutri/hung_resp.pkl', 'rb') as f:
    hung_resp = pickle.load(f)

In [ ]:
trans_hung_resp = translate_to_eng(hung_resp)
trans_hung_resp

In [ ]:
hung_df = parse_to_dataframe(trans_hung_resp)

hung_df

In [ ]:
hung_resp2 = call_openai_api(user_prompt_hung, system_prompt_hung, 5)
hung_resp2

In [ ]:
# with open('./responses/cost_nutri/hung_resp2.pkl', 'wb') as f:
#     pickle.dump(hung_resp2, f)

In [ ]:
with open('./responses/cost_nutri/hung_resp2.pkl', 'rb') as f:
    hung_resp2 = pickle.load(f)

In [ ]:
trans_hung_resp2 = translate_to_eng(hung_resp2)
trans_hung_resp2

In [ ]:
hung_df2 = parse_to_dataframe(trans_hung_resp2)

hung_df2

In [ ]:
hung_df_final = pd.concat([hung_df, hung_df2[:20]], ignore_index=True)
hung_df_final

In [ ]:
hung_df_final.to_pickle("../dataset/cost_nutri/hung_result.pkl")

### Collect Irish Version

In [ ]:
user_prompt_irish = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_irish.txt')
system_prompt_irish = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_irish.txt')

In [ ]:
irish_resp = call_openai_api(user_prompt_irish, system_prompt_irish, 35)
irish_resp

In [ ]:
with open('./responses/cost_nutri/irish_resp.pkl', 'wb') as f:
    pickle.dump(irish_resp, f)

In [ ]:
with open('./responses/cost_nutri/irish_resp.pkl', 'rb') as f:
    irish_resp = pickle.load(f)

In [ ]:
trans_irish_resp = translate_to_eng(irish_resp)
trans_irish_resp

In [ ]:
irish_df = parse_to_dataframe(trans_irish_resp)

irish_df

In [ ]:
irish_resp2 = call_openai_api(user_prompt_irish, system_prompt_irish, 5)
irish_resp2

In [ ]:
trans_irish_resp2 = translate_to_eng(irish_resp2)
trans_irish_resp2

In [ ]:
irish_df2 = parse_to_dataframe(trans_irish_resp2)

irish_df2

In [ ]:
irish_df_final = pd.concat([irish_df, irish_df2[:21]], ignore_index=True)
irish_df_final

In [ ]:
irish_df_final.to_pickle("../dataset/cost_nutri/irish_result.pkl")

### Collect Italian Version

In [ ]:
user_prompt_italian = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_italian.txt')
system_prompt_italian = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_italian.txt')

In [ ]:
italian_resp = call_openai_api(user_prompt_italian, system_prompt_italian, 35)
italian_resp

In [ ]:
with open('./responses/cost_nutri/italian_resp.pkl', 'wb') as f:
    pickle.dump(italian_resp, f)

In [ ]:
with open('./responses/cost_nutri/italian_resp.pkl', 'rb') as f:
    italian_resp = pickle.load(f)

In [ ]:
trans_italian_resp = translate_to_eng(italian_resp)
trans_italian_resp

In [ ]:
italian_df = parse_to_dataframe(trans_italian_resp)

italian_df

In [ ]:
italian_df_final = italian_df[:300]
italian_df_final

In [ ]:
italian_df_final.to_pickle("../dataset/cost_nutri/italian_result.pkl")

### Collect Japanese Version

In [ ]:
user_prompt_jap = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_jap.txt')
system_prompt_jap = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_jap.txt')

In [ ]:
jap_resp = call_openai_api(user_prompt_jap, system_prompt_jap, 35)
jap_resp

In [ ]:
with open('./responses/cost_nutri/jap_resp.pkl', 'wb') as f:
    pickle.dump(jap_resp, f)

In [ ]:
with open('./responses/cost_nutri/jap_resp.pkl', 'rb') as f:
    jap_resp = pickle.load(f)

In [ ]:
trans_jap_resp = translate_to_eng(jap_resp)
trans_jap_resp

In [ ]:
jap_df = parse_to_dataframe(trans_jap_resp)

jap_df

In [ ]:
jap_resp2 = call_openai_api(user_prompt_jap, system_prompt_jap, 2)
jap_resp2

In [ ]:
# with open('./responses/cost_nutri/jap_resp2.pkl', 'wb') as f:
#     pickle.dump(jap_resp2, f)

In [ ]:
with open('./responses/cost_nutri/jap_resp2.pkl', 'rb') as f:
    jap_resp2 = pickle.load(f)

In [ ]:
trans_jap_resp2 = translate_to_eng(jap_resp2)
trans_jap_resp2

In [ ]:
jap_df2 = parse_to_dataframe(trans_jap_resp2)

jap_df2

In [ ]:
jap_df_final = pd.concat([jap_df, jap_df2[:7]], ignore_index=True)
jap_df_final

In [ ]:
jap_df_final.to_pickle("../dataset/cost_nutri/jap_result.pkl")


### Collect Korean Version

In [ ]:
user_prompt_kor = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_korean.txt')
system_prompt_kor = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_korean.txt')

In [ ]:
kor_resp = call_openai_api(user_prompt_kor, system_prompt_kor, 35)
kor_resp

In [ ]:
# with open('./responses/cost_nutri/kor_resp.pkl', 'wb') as f:
#     pickle.dump(kor_resp, f)

In [ ]:
with open('./responses/cost_nutri/kor_resp.pkl', 'rb') as f:
    kor_resp = pickle.load(f)

In [ ]:
trans_kor_resp = translate_to_eng(kor_resp)
trans_kor_resp

In [ ]:
kor_df = parse_to_dataframe(trans_kor_resp)

kor_df

In [ ]:
kor_df_final = kor_df[:300]
kor_df_final

In [ ]:
kor_df_final.to_pickle("../dataset/cost_nutri/kor_result.pkl")


### Collect Portugese Version

In [ ]:
user_prompt_port = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_port.txt')
system_prompt_port = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_port.txt')

In [ ]:
port_resp = call_openai_api(user_prompt_port, system_prompt_port, 35)
port_resp

In [ ]:
# with open('./responses/cost_nutri/port_resp.pkl', 'wb') as f:
#     pickle.dump(port_resp, f)

In [ ]:
with open('./responses/cost_nutri/port_resp.pkl', 'rb') as f:
    port_resp = pickle.load(f)

In [ ]:
trans_port_resp = translate_to_eng(port_resp)
trans_port_resp

In [ ]:
port_df = parse_to_dataframe(trans_port_resp)

port_df

In [ ]:
port_df_final = port_df[:300]
port_df_final

In [ ]:
port_df_final.to_pickle("../dataset/cost_nutri/port_result.pkl")



### Collect Russian Version

In [ ]:
user_prompt_russian = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_russian.txt')
system_prompt_russian = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_russian.txt')

In [ ]:
russian_resp = call_openai_api(user_prompt_russian, system_prompt_russian, 35)
russian_resp

In [ ]:
# with open('./responses/cost_nutri/russian_resp.pkl', 'wb') as f:
#     pickle.dump(russian_resp, f)

In [ ]:
with open('./responses/cost_nutri/russian_resp.pkl', 'rb') as f:
    russian_resp = pickle.load(f)

In [ ]:
trans_russian_resp = translate_to_eng(russian_resp)
trans_russian_resp

In [ ]:
russian_df = parse_to_dataframe(trans_russian_resp)

russian_df

In [ ]:
russian_resp2 = call_openai_api(user_prompt_russian, system_prompt_russian, 10)
russian_resp2

In [ ]:
# with open('./responses/cost_nutri/russian_resp2.pkl', 'wb') as f:
#     pickle.dump(russian_resp2, f)

In [ ]:
with open('./responses/cost_nutri/russian_resp2.pkl', 'rb') as f:
    russian_resp2 = pickle.load(f)

In [ ]:
trans_russian_resp2 = translate_to_eng(russian_resp2)
trans_russian_resp2

In [ ]:
russian_df2 = parse_to_dataframe(trans_russian_resp2)

russian_df2

In [ ]:
russian_df_final = pd.concat([russian_df, russian_df2[:64]], ignore_index=True)
russian_df_final

In [ ]:
russian_df_final.to_pickle("../dataset/cost_nutri/russian_result.pkl")



### Collect Spanish Version

In [14]:
user_prompt_spanish = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_spanish.txt')
system_prompt_spanish = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_spanish.txt')

In [ ]:
spanish_resp = call_openai_api(user_prompt_spanish, system_prompt_spanish, 35)
spanish_resp

In [ ]:
# with open('./responses/cost_nutri/spanish_resp.pkl', 'wb') as f:
#     pickle.dump(spanish_resp, f)

In [8]:
with open('./responses/cost_nutri/spanish_resp.pkl', 'rb') as f:
    spanish_resp = pickle.load(f)

In [9]:
spanish_resp

["Aquí tienes 10 recetas saludables con sus costos y valores nutricionales por porción:\n\n[{'nombre': 'ensalada de quinoa',\n  'costos': 3,\n  'calorías': 250,\n  'proteína': 8,\n  'gordo': 7,\n  'carbohidratos': 39},\n {'nombre': 'smoothie de espinacas y plátano',\n  'costos': 2,\n  'calorías': 180,\n  'proteína': 5,\n  'gordo': 1,\n  'carbohidratos': 36},\n {'nombre': 'avena nocturna',\n  'costos': 1,\n  'calorías': 215,\n  'proteína': 6,\n  'gordo': 3.5,\n  'carbohidratos': 40},\n {'nombre': 'sopa de lentejas',\n  'costos': 2,\n  'calorías': 230,\n  'proteína': 18,\n  'gordo': 0.8,\n  'carbohidratos': 40},\n {'nombre': 'bowl de buddha',\n  'costos': 4,\n  'calorías': 410,\n  'proteína': 12,\n  'gordo': 14,\n  'carbohidratos': 65},\n {'nombre': 'ensalada de garbanzos',\n  'costos': 3,\n  'calorías': 350,\n  'proteína': 15,\n  'gordo': 12,\n  'carbohidratos': 50},\n {'nombre': 'wrap de hummus y verduras',\n  'costos': 3,\n  'calorías': 290,\n  'proteína': 10,\n  'gordo': 9,\n  'carbo

In [10]:
trans_spanish_resp = translate_to_eng(spanish_resp)
trans_spanish_resp

["here are 10 healthy recipes with their costs and nutritional values \u200b\u200bper serving:\n\n[{'name': 'quinoa salad',\n  'costs': 3,\n  'calories': 250,\n  'protein': 8,\n  'fat': 7,\n  'carbohydrates': 39},\n {'name': 'banana spinach smoothie',\n  'costs': 2,\n  'calories': 180,\n  'protein': 5,\n  'fat': 1,\n  'carbohydrates': 36},\n {'name': 'night oats',\n  'costs': 1,\n  'calories': 215,\n  'protein': 6,\n  'fat': 3.5,\n  'carbohydrates': 40},\n {'name': 'lentil soup',\n  'costs': 2,\n  'calories': 230,\n  'protein': 18,\n  'fat': 0.8,\n  'carbohydrates': 40},\n {'name': 'buddha bowl',\n  'costs': 4,\n  'calories': 410,\n  'protein': 12,\n  'fat': 14,\n  'carbohydrates': 65},\n {'name': 'chickpea salad',\n  'costs': 3,\n  'calories': 350,\n  'protein': 15,\n  'fat': 12,\n  'carbohydrates': 50},\n {'name': 'hummus and veggie wrap',\n  'costs': 3,\n  'calories': 290,\n  'protein': 10,\n  'fat': 9,\n  'carbohydrates': 45},\n {'name': 'spring pasta',\n  'costs': 4,\n  'calories'

In [11]:
spanish_df = parse_to_dataframe(trans_spanish_resp)

spanish_df

Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Error decoding JSON: Expecting ',' delimiter: line 68 column 11 (char 1179)
Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Error decoding JSON: Expecting value: line 1 column 1 (char 0)


,name,costs,calorie,protein,fat,carbohydrates
0,quinoa and chickpea salad,4.0,350,15,10.0,50
1,banana spinach smoothie,3.0,220,5,2.0,45
2,lentil soup,3.0,300,18,5.0,40
3,kale and avocado salad,5.0,400,8,25.0,30
4,tofu and vegetable wrap,6.0,350,21,10.0,35
...,...,...,...,...,...,...
254,mediterranean chickpea salad,5.0,270,12,14.0,30
255,berry and spinach smoothie bowl,4.0,230,6,4.0,45
256,bowl of tofu and mixed vegetables,6.0,320,20,16.0,28
257,quinoa salad with avocado,5.0,290,10,18.0,28


In [15]:
spanish_resp2 = call_openai_api(user_prompt_spanish, system_prompt_spanish, 7)
spanish_resp2

[{'nombre': 'ensalada de quinoa y vegetales',
  'costos': 4,
  'calorías': 250,
  'proteína': 8,
  'gordo': 7,
  'carbohidratos': 40},
 {'nombre': 'batido de espinacas y plátano',
  'costos': 3,
  'calorías': 180,
  'proteína': 5,
  'gordo': 1,
  'carbohidratos': 35},
 {'nombre': 'sopa de lentejas',
  'costos': 3,
  'calorías': 230,
  'proteína': 15,
  'gordo': 1,
  'carbohidratos': 40},
 {'nombre': 'pollo al vapor con brócoli',
  'costos': 5,
  'calorías': 310,
  'proteína': 35,
  'gordo': 5,
  'carbohidratos': 10},
 {'nombre': 'ensalada de garbanzos con tomate',
  'costos': 4,
  'calorías': 290,
  'proteína': 15,
  'gordo': 8,
  'carbohidratos': 45},
 {'nombre': 'tocino de pavo con aguacate',
  'costos': 6,
  'calorías': 270,
  'proteína': 20,
  'gordo': 15,
  'carbohidratos': 12},
 {'nombre': 'omelette de claras de huevo y espinacas',
  'costos': 4,
  'calorías': 140,
  'proteína': 20,
  'gordo': 5,
  'carbohidratos': 3},
 {'nombre': 'smoothie de arándanos y yogurt griego',
  'costo

["[{'nombre': 'ensalada de quinoa y vegetales',\n  'costos': 4,\n  'calorías': 250,\n  'proteína': 8,\n  'gordo': 7,\n  'carbohidratos': 40},\n {'nombre': 'batido de espinacas y plátano',\n  'costos': 3,\n  'calorías': 180,\n  'proteína': 5,\n  'gordo': 1,\n  'carbohidratos': 35},\n {'nombre': 'sopa de lentejas',\n  'costos': 3,\n  'calorías': 230,\n  'proteína': 15,\n  'gordo': 1,\n  'carbohidratos': 40},\n {'nombre': 'pollo al vapor con brócoli',\n  'costos': 5,\n  'calorías': 310,\n  'proteína': 35,\n  'gordo': 5,\n  'carbohidratos': 10},\n {'nombre': 'ensalada de garbanzos con tomate',\n  'costos': 4,\n  'calorías': 290,\n  'proteína': 15,\n  'gordo': 8,\n  'carbohidratos': 45},\n {'nombre': 'tocino de pavo con aguacate',\n  'costos': 6,\n  'calorías': 270,\n  'proteína': 20,\n  'gordo': 15,\n  'carbohidratos': 12},\n {'nombre': 'omelette de claras de huevo y espinacas',\n  'costos': 4,\n  'calorías': 140,\n  'proteína': 20,\n  'gordo': 5,\n  'carbohidratos': 3},\n {'nombre': 'smoo

In [16]:
# with open('./responses/cost_nutri/spanish_resp2.pkl', 'wb') as f:
#     pickle.dump(spanish_resp2, f)

In [18]:
with open('./responses/cost_nutri/spanish_resp2.pkl', 'rb') as f:
    spanish_resp2 = pickle.load(f)

In [19]:
trans_spanish_resp2 = translate_to_eng(spanish_resp2)
trans_spanish_resp2

["[{'name': 'quinoa and vegetable salad',\n  'costs': 4,\n  'calories': 250,\n  'protein': 8,\n  'fat': 7,\n  'carbohydrates': 40},\n {'name': 'banana spinach smoothie',\n  'costs': 3,\n  'calories': 180,\n  'protein': 5,\n  'fat': 1,\n  'carbohydrates': 35},\n {'name': 'lentil soup',\n  'costs': 3,\n  'calories': 230,\n  'protein': 15,\n  'fat': 1,\n  'carbohydrates': 40},\n {'name': 'steamed chicken with broccoli',\n  'costs': 5,\n  'calories': 310,\n  'protein': 35,\n  'fat': 5,\n  'carbohydrates': 10},\n {'name': 'chickpea salad with tomato',\n  'costs': 4,\n  'calories': 290,\n  'protein': 15,\n  'fat': 8,\n  'carbohydrates': 45},\n {'name': 'turkey bacon with avocado',\n  'costs': 6,\n  'calories': 270,\n  'protein': 20,\n  'fat': 15,\n  'carbohydrates': 12},\n {'name': 'egg white and spinach omelet',\n  'costs': 4,\n  'calories': 140,\n  'protein': 20,\n  'fat': 5,\n  'carbohydrates': 3},\n {'name': 'blueberry greek yogurt smoothie',\n  'costs': 3,\n  'calories': 190,\n  'protei

In [20]:
spanish_df2 = parse_to_dataframe(trans_spanish_resp2)

spanish_df2

Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Error decoding JSON: Expecting value: line 1 column 1 (char 0)


,name,costs,calorie,protein,fat,carbohydrates
0,quinoa and vegetable salad,4.0,250,8,7.0,40
1,banana spinach smoothie,3.0,180,5,1.0,35
2,lentil soup,3.0,230,15,1.0,40
3,steamed chicken with broccoli,5.0,310,35,5.0,10
4,chickpea salad with tomato,4.0,290,15,8.0,45
5,turkey bacon with avocado,6.0,270,20,15.0,12
6,egg white and spinach omelet,4.0,140,20,5.0,3
7,blueberry greek yogurt smoothie,3.0,190,10,2.0,35
8,tofu and vegetable stir fry,4.0,255,18,12.0,20
9,fried acamaya with garlic,8.0,300,25,18.0,5


In [27]:
spanish_resp3 = call_openai_api(user_prompt_spanish, system_prompt_spanish, 4)
spanish_resp3

Aquí tienes 10 recetas saludables con sus respectivos costos en USD y valores nutricionales por porción:

[{'nombre': 'Ensalada de espinacas y fresas',
  'costos': 4,
  'calorías': 150,
  'proteína': 4,
  'gordo': 10,
  'carbohidratos': 16},
 {'nombre': 'Batido de kale y plátano',
  'costos': 3,
  'calorías': 180,
  'proteína': 5,
  'gordo': 2,
  'carbohidratos': 35},
 {'nombre': 'Quinoa con verduras al vapor',
  'costos': 6,
  'calorías': 250,
  'proteína': 9,
  'gordo': 5,
  'carbohidratos': 45},
 {'nombre': 'Salmón al horno con limón',
  'costos': 8,
  'calorías': 300,
  'proteína': 23,
  'gordo': 15,
  'carbohidratos': 0},
 {'nombre': 'Pollo al curry con arroz integral',
  'costos': 7,
  'calorías': 350,
  'proteína': 26,
  'gordo': 10,
  'carbohidratos': 40},
 {'nombre': 'Sopa de lentejas',
  'costos': 3,
  'calorías': 220,
  'proteína': 14,
  'gordo': 5,
  'carbohidratos': 30},
 {'nombre': 'Tacos de pescado',
  'costos': 8,
  'calorías': 270,
  'proteína': 18,
  'gordo': 9,
  'ca

["Aquí tienes 10 recetas saludables con sus respectivos costos en USD y valores nutricionales por porción:\n\n[{'nombre': 'Ensalada de espinacas y fresas',\n  'costos': 4,\n  'calorías': 150,\n  'proteína': 4,\n  'gordo': 10,\n  'carbohidratos': 16},\n {'nombre': 'Batido de kale y plátano',\n  'costos': 3,\n  'calorías': 180,\n  'proteína': 5,\n  'gordo': 2,\n  'carbohidratos': 35},\n {'nombre': 'Quinoa con verduras al vapor',\n  'costos': 6,\n  'calorías': 250,\n  'proteína': 9,\n  'gordo': 5,\n  'carbohidratos': 45},\n {'nombre': 'Salmón al horno con limón',\n  'costos': 8,\n  'calorías': 300,\n  'proteína': 23,\n  'gordo': 15,\n  'carbohidratos': 0},\n {'nombre': 'Pollo al curry con arroz integral',\n  'costos': 7,\n  'calorías': 350,\n  'proteína': 26,\n  'gordo': 10,\n  'carbohidratos': 40},\n {'nombre': 'Sopa de lentejas',\n  'costos': 3,\n  'calorías': 220,\n  'proteína': 14,\n  'gordo': 5,\n  'carbohidratos': 30},\n {'nombre': 'Tacos de pescado',\n  'costos': 8,\n  'calorías': 

In [28]:
with open('./responses/cost_nutri/spanish_resp3.pkl', 'wb') as f:
    pickle.dump(spanish_resp3, f)

In [29]:
with open('./responses/cost_nutri/spanish_resp3.pkl', 'rb') as f:
    spanish_resp3 = pickle.load(f)

In [30]:
trans_spanish_resp3 = translate_to_eng(spanish_resp3)
trans_spanish_resp3

["here are 10 healthy recipes with their respective costs in usd and nutritional values \u200b\u200bper serving:\n\n[{'name': 'strawberry spinach salad',\n  'costs': 4,\n  'calories': 150,\n  'protein': 4,\n  'fat': 10,\n  'carbohydrates': 16},\n {'name': 'kale banana smoothie',\n  'costs': 3,\n  'calories': 180,\n  'protein': 5,\n  'fat': 2,\n  'carbohydrates': 35},\n {'name': 'quinoa with steamed vegetables',\n  'costs': 6,\n  'calories': 250,\n  'protein': 9,\n  'fat': 5,\n  'carbohydrates': 45},\n {'name': 'baked salmon with lemon',\n  'costs': 8,\n  'calories': 300,\n  'protein': 23,\n  'fat': 15,\n  'carbohydrates': 0},\n {'name': 'chicken curry with brown rice',\n  'costs': 7,\n  'calories': 350,\n  'protein': 26,\n  'fat': 10,\n  'carbohydrates': 40},\n {'name': 'lentil soup',\n  'costs': 3,\n  'calories': 220,\n  'protein': 14,\n  'fat': 5,\n  'carbohydrates': 30},\n {'name': 'fish tacos',\n  'costs': 8,\n  'calories': 270,\n  'protein': 18,\n  'fat': 9,\n  'carbohydrates': 30

In [31]:
spanish_df3 = parse_to_dataframe(trans_spanish_resp3)

spanish_df3

Error decoding JSON: Expecting value: line 1 column 1 (char 0)


,name,costs,calorie,protein,fat,carbohydrates
0,quinoa and vegetable salad,4.0,300,8,10,45
1,banana spinach smoothie,3.0,250,5,3,50
2,lentil soup,3.0,220,18,3,30
3,baked eggs with spinach,5.0,180,12,8,5
4,lime fish tacos,6.0,350,25,9,40
5,chickpea salad with tomato and cucumber,4.0,270,9,6,40
6,roasted chicken with broccoli,7.0,310,35,15,10
7,baked salmon with asparagus,10.0,400,34,22,12
8,carrot cream,2.0,140,2,5,22
9,tofu and mango salad,5.0,250,14,12,28


In [26]:
spanish_df_final = pd.concat([spanish_df, spanish_df2], ignore_index=True)
spanish_df_final

,name,costs,calorie,protein,fat,carbohydrates
0,quinoa and chickpea salad,4.0,350,15,10.0,50
1,banana spinach smoothie,3.0,220,5,2.0,45
2,lentil soup,3.0,300,18,5.0,40
3,kale and avocado salad,5.0,400,8,25.0,30
4,tofu and vegetable wrap,6.0,350,21,10.0,35
...,...,...,...,...,...,...
284,oatmeal with banana and almonds,2.0,310,10,8.0,56
285,quinoa salad with chickpeas,4.0,340,13,9.0,55
286,chicken and avocado wrap,5.0,380,25,15.0,35
287,baked salmon with vegetables,7.0,500,35,30.0,15


In [32]:
spanish_df_final = pd.concat([spanish_df_final, spanish_df3[:11]], ignore_index=True)
spanish_df_final

,name,costs,calorie,protein,fat,carbohydrates
0,quinoa and chickpea salad,4.0,350,15,10.0,50
1,banana spinach smoothie,3.0,220,5,2.0,45
2,lentil soup,3.0,300,18,5.0,40
3,kale and avocado salad,5.0,400,8,25.0,30
4,tofu and vegetable wrap,6.0,350,21,10.0,35
...,...,...,...,...,...,...
295,roasted chicken with broccoli,7.0,310,35,15.0,10
296,baked salmon with asparagus,10.0,400,34,22.0,12
297,carrot cream,2.0,140,2,5.0,22
298,tofu and mango salad,5.0,250,14,12.0,28


In [33]:
spanish_df_final.to_pickle("../dataset/cost_nutri/spanish_result.pkl")

### Collect Swedish Version

In [34]:
user_prompt_swedish = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_swedish.txt')
system_prompt_swedish = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_swedish.txt')

In [36]:
swedish_resp = call_openai_api(user_prompt_swedish, system_prompt_swedish, 35)
swedish_resp

[{'name': 'Quinoasallad',
  'kostnader': 4,
  'kalori': 350,
  'protein': 8,
  "fett": 10,
  'kolhydrater': 45},
 {'name': 'Grönsakssoppa',
  'kostnader': 3,
  'kalori': 200,
  'protein': 5,
  "fett": 5,
  'kolhydrater': 30},
 {'name': 'Kyckling och broccoli',
  'kostnader': 6,
  'kalori': 400,
  'protein': 35,
  "fett": 10,
  'kolhydrater': 10},
 {'name': 'Lax med spenat',
  'kostnader': 10,
  'kalori': 500,
  'protein': 30,
  "fett": 25,
  'kolhydrater': 5},
 {'name': 'Grillade grönsaksspett',
  'kostnader': 5,
  'kalori': 150,
  'protein': 5,
  "fett": 8,
  'kolhydrater': 15},
 {'wame': 'Gröt med blåbär',
  'kostnader': 2,
  'kalori': 250,
  'protein': 6,
  "fett": 4,
  'kolhydrater': 40},
 {'name': 'Kikärtscurry',
  'kostnader': 3,
  'kalori': 300,
  'protein': 9,
  "fett": 10,
  'kolhydrater': 35},
 {'name': 'Hummustallrik med grönsaker',
  'kostnader': 4,
  'kalori': 350,
  'protein': 12,
  "fett": 15,
  'kolhydrater': 30},
 {'name': 'Turkisk yoghurt med honung och valnötter',
  

['[{\'name\': \'Quinoasallad\',\n  \'kostnader\': 4,\n  \'kalori\': 350,\n  \'protein\': 8,\n  "fett": 10,\n  \'kolhydrater\': 45},\n {\'name\': \'Grönsakssoppa\',\n  \'kostnader\': 3,\n  \'kalori\': 200,\n  \'protein\': 5,\n  "fett": 5,\n  \'kolhydrater\': 30},\n {\'name\': \'Kyckling och broccoli\',\n  \'kostnader\': 6,\n  \'kalori\': 400,\n  \'protein\': 35,\n  "fett": 10,\n  \'kolhydrater\': 10},\n {\'name\': \'Lax med spenat\',\n  \'kostnader\': 10,\n  \'kalori\': 500,\n  \'protein\': 30,\n  "fett": 25,\n  \'kolhydrater\': 5},\n {\'name\': \'Grillade grönsaksspett\',\n  \'kostnader\': 5,\n  \'kalori\': 150,\n  \'protein\': 5,\n  "fett": 8,\n  \'kolhydrater\': 15},\n {\'wame\': \'Gröt med blåbär\',\n  \'kostnader\': 2,\n  \'kalori\': 250,\n  \'protein\': 6,\n  "fett": 4,\n  \'kolhydrater\': 40},\n {\'name\': \'Kikärtscurry\',\n  \'kostnader\': 3,\n  \'kalori\': 300,\n  \'protein\': 9,\n  "fett": 10,\n  \'kolhydrater\': 35},\n {\'name\': \'Hummustallrik med grönsaker\',\n  \'kostnad

In [37]:
# with open('./responses/cost_nutri/swedish_resp.pkl', 'wb') as f:
#     pickle.dump(swedish_resp, f)

In [38]:
with open('./responses/cost_nutri/swedish_resp.pkl', 'rb') as f:
    swedish_resp = pickle.load(f)

In [39]:
trans_swedish_resp = translate_to_eng(swedish_resp)
trans_swedish_resp

['[{\'name\': \'quinoa salad\',\n  \'costs\': 4,\n  \'calorie\': 350,\n  \'protein\': 8,\n  "fat": 10,\n  \'carbs\': 45},\n {\'name\': \'vegetable soup\',\n  \'costs\': 3,\n  \'calorie\': 200,\n  \'protein\': 5,\n  "fat": 5,\n  \'carbs\': 30},\n {\'name\': \'chicken and broccoli\',\n  \'costs\': 6,\n  \'calorie\': 400,\n  \'protein\': 35,\n  "fat": 10,\n  \'carbs\': 10},\n {\'name\': \'salmon with spinach\',\n  \'costs\': 10,\n  \'calorie\': 500,\n  \'protein\': 30,\n  "fat": 25,\n  \'carbs\': 5},\n {\'name\': \'grilled vegetable skewers\',\n  \'costs\': 5,\n  \'calorie\': 150,\n  \'protein\': 5,\n  "fat": 8,\n  \'carbs\': 15},\n {\'wame\': \'porridge with blueberries\',\n  \'costs\': 2,\n  \'calorie\': 250,\n  \'protein\': 6,\n  "fat": 4,\n  \'carbs\': 40},\n {\'name\': \'chickpea curry\',\n  \'costs\': 3,\n  \'calorie\': 300,\n  \'protein\': 9,\n  "fat": 10,\n  \'carbs\': 35},\n {\'name\': \'hummus plate with vegetables\',\n  \'costs\': 4,\n  \'calorie\': 350,\n  \'protein\': 12,\n  

In [40]:
swedish_df = parse_to_dataframe(trans_swedish_resp)

swedish_df

Error decoding JSON: Expecting value: line 1 column 1 (char 0)


,name,costs,calorie,protein,fat,carbohydrates
0,quinoa salad,4.0,350,8,10.0,45
1,vegetable soup,3.0,200,5,5.0,30
2,chicken and broccoli,6.0,400,35,10.0,10
3,salmon with spinach,10.0,500,30,25.0,5
4,grilled vegetable skewers,5.0,150,5,8.0,15
...,...,...,...,...,...,...
320,chickpea curry,4.0,400,14,18.0,45
321,avocado and scrambled eggs,4.0,270,13,20.0,5
322,spinach and potato salad,3.0,150,4,5.0,23
323,carrot soup,2.0,120,2,3.0,22


In [41]:
swedish_df_final = swedish_df[:300]
swedish_df_final

,name,costs,calorie,protein,fat,carbohydrates
0,quinoa salad,4.00,350,8,10.0,45
1,vegetable soup,3.00,200,5,5.0,30
2,chicken and broccoli,6.00,400,35,10.0,10
3,salmon with spinach,10.00,500,30,25.0,5
4,grilled vegetable skewers,5.00,150,5,8.0,15
...,...,...,...,...,...,...
295,quinoa salad,4.50,320,8,7.0,55
296,vegetable soup,3.75,180,5,1.0,35
297,broccoli and tofu stir fry,6.00,250,12,10.0,18
298,chickpea curry,4.00,400,13,10.0,60


In [42]:
swedish_df_final.to_pickle("../dataset/cost_nutri/swedish_result.pkl")

### Collect Thai Version

In [43]:
user_prompt_thai = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_thai.txt')
system_prompt_thai = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_thai.txt')

In [45]:
thai_resp = call_openai_api(user_prompt_thai, system_prompt_thai, 35)
thai_resp

นี่คือค่าใช้จ่ายและคุณค่าทางโภชนาการของสูตรอาหารเพื่อสุขภาพ 10 รายการ:

[{'name': 'สลัดผัก',
  'ต้นทุน': 4,
  'แคลอรี่': 150,
  'โปรตีน': 5,
  'อ้วน': 10,
  'คาร์โบไฮเดรต': 18},
 {'name': 'สมูทตี้ผลไม้',
  'ต้นทุน': 3,
  'แคลอรี่': 180,
  'โปรตีน': 2,
  'อ้วน': 0,
  'คาร์โบไฮเดรต': 40},
 {'name': 'โจ๊กข้าวโอ๊ต',
  'ต้นทุน': 2,
  'แคลอรี่': 200,
  'โปรตีน': 6,
  'อ้วน': 3,
  'คาร์โบไฮเดรต': 38},
 {'name': 'แพนเค้กข้าวโอ๊ต',
  'ต้นทุน': 4,
  'แคลอรี่': 250,
  'โปรตีน': 8,
  'อ้วน': 7,
  'คาร์โบไฮเดรต': 30},
 {'name': 'กล้วยอบแห้ง',
  'ต้นทุน': 1,
  'แคลอรี่': 300,
  'โปรตีน': 3,
  'อ้วน': 0,
  'คาร์โบไฮเดรต': 70},
 {'name': 'ซุปข้าวกล้อง',
  'ต้นทุน': 3,
  'แคลอรี่': 150,
  'โปรตีน': 4,
  'อ้วน': 1,
  'คาร์โบไฮเดรต': 30},
 {'name': 'ไส้กรอกเจไทย',
  'ต้นทุน': 2,
  'แคลอรี่': 220,
  'โปรตีน': 12,
  'อ้วน': 15,
  'คาร์โบไฮเดรต': 10},
 {'name': 'เต้าหู้ย่าง',
  'ต้นทุน': 2,
  'แคลอรี่': 100,
  'โปรตีน': 8,
  'อ้วน': 4,
  'คาร์โบไฮเดรต': 2},
 {'name': 'ยำเห็ด',
  'ต้นทุน': 4,
  'แคลอรี่': 12

["นี่คือค่าใช้จ่ายและคุณค่าทางโภชนาการของสูตรอาหารเพื่อสุขภาพ 10 รายการ:\n\n[{'name': 'สลัดผัก',\n  'ต้นทุน': 4,\n  'แคลอรี่': 150,\n  'โปรตีน': 5,\n  'อ้วน': 10,\n  'คาร์โบไฮเดรต': 18},\n {'name': 'สมูทตี้ผลไม้',\n  'ต้นทุน': 3,\n  'แคลอรี่': 180,\n  'โปรตีน': 2,\n  'อ้วน': 0,\n  'คาร์โบไฮเดรต': 40},\n {'name': 'โจ๊กข้าวโอ๊ต',\n  'ต้นทุน': 2,\n  'แคลอรี่': 200,\n  'โปรตีน': 6,\n  'อ้วน': 3,\n  'คาร์โบไฮเดรต': 38},\n {'name': 'แพนเค้กข้าวโอ๊ต',\n  'ต้นทุน': 4,\n  'แคลอรี่': 250,\n  'โปรตีน': 8,\n  'อ้วน': 7,\n  'คาร์โบไฮเดรต': 30},\n {'name': 'กล้วยอบแห้ง',\n  'ต้นทุน': 1,\n  'แคลอรี่': 300,\n  'โปรตีน': 3,\n  'อ้วน': 0,\n  'คาร์โบไฮเดรต': 70},\n {'name': 'ซุปข้าวกล้อง',\n  'ต้นทุน': 3,\n  'แคลอรี่': 150,\n  'โปรตีน': 4,\n  'อ้วน': 1,\n  'คาร์โบไฮเดรต': 30},\n {'name': 'ไส้กรอกเจไทย',\n  'ต้นทุน': 2,\n  'แคลอรี่': 220,\n  'โปรตีน': 12,\n  'อ้วน': 15,\n  'คาร์โบไฮเดรต': 10},\n {'name': 'เต้าหู้ย่าง',\n  'ต้นทุน': 2,\n  'แคลอรี่': 100,\n  'โปรตีน': 8,\n  'อ้วน': 4,\n  'คาร์โบไฮเดรต': 2},

In [46]:
# with open('./responses/cost_nutri/thai_resp.pkl', 'wb') as f:
#     pickle.dump(thai_resp, f)

In [47]:
with open('./responses/cost_nutri/thai_resp.pkl', 'rb') as f:
    thai_resp = pickle.load(f)

In [48]:
trans_thai_resp = translate_to_eng(thai_resp)
trans_thai_resp

["here are the costs and nutritional values \u200b\u200bof 10 healthy recipes:\n\n[{'name': 'vegetable salad',\n  'cost': 4,\n  'calories': 150,\n  'protein': 5,\n  'fat': 10,\n  'carbohydrates': 18},\n {'name': 'fruit smoothie',\n  'cost': 3,\n  'calories': 180,\n  'protein': 2,\n  'fat': 0,\n  'carbohydrates': 40},\n {'name': 'oat porridge',\n  'cost': 2,\n  'calories': 200,\n  'protein': 6,\n  'fat': 3,\n  'carbohydrates': 38},\n {'name': 'oatmeal pancake',\n  'cost': 4,\n  'calories': 250,\n  'protein': 8,\n  'fat': 7,\n  'carbohydrates': 30},\n {'name': 'dried banana',\n  'cost': 1,\n  'calories': 300,\n  'protein': 3,\n  'fat': 0,\n  'carbohydrates': 70},\n {'name': 'brown rice soup',\n  'cost': 3,\n  'calories': 150,\n  'protein': 4,\n  'fat': 1,\n  'carbohydrates': 30},\n {'name': 'thai vegetarian sausage',\n  'cost': 2,\n  'calories': 220,\n  'protein': 12,\n  'fat': 15,\n  'carbohydrates': 10},\n {'name': 'grilled tofu',\n  'cost': 2,\n  'calories': 100,\n  'protein': 8,\n  '

In [66]:
thai_df = parse_to_dataframe(trans_thai_resp)

thai_df

Error decoding JSON: Extra data: line 62 column 1 (char 1189) in string here is a list of 10 healthy recipes, including the cost and nutritional value of each recipe:

[{'n...
Error decoding JSON: Extra data: line 72 column 1 (char 1215) in string here are 10 healthy recipes with cost and nutritional details per serving:

[{'name': 'fresh vegetab...
Error decoding JSON: Extra data: line 8 column 1 (char 114) in string here is a list of 10 healthy recipes with cost and nutritional value per serving:

1. [{'name': 'veg...
Error decoding JSON: Expecting value: line 1 column 1 (char 0) in string here is a list of 10 healthy recipes, along with each recipe's cost and nutritional value per servin...
Error decoding JSON: Expecting value: line 1 column 1 (char 0) in string here are 10 healthy recipes you can try at home:

1. grilled chicken salad
2. grilled salmon with ve...
Error decoding JSON: Extra data: line 61 column 1 (char 1137) in string here is a dataset of 10 healthy recipes, with th

,name,cost,calories,protein,fat,carbohydrates
0,vegetable salad,4.0,150,5,10.0,18
1,fruit smoothie,3.0,180,2,0.0,40
2,oat porridge,2.0,200,6,3.0,38
3,oatmeal pancake,4.0,250,8,7.0,30
4,dried banana,1.0,300,3,0.0,70
...,...,...,...,...,...,...
265,crispy coriander leaves,1.5,100,3,6.0,9
266,salmon steak,6.0,400,32,28.0,0
267,tofu salad,2.8,220,12,14.0,12
268,stir-fried quinoa tom yum,4.0,340,10,12.0,50


In [67]:
thai_resp2 = call_openai_api(user_prompt_thai, system_prompt_thai, 5)
thai_resp2

นี่คือรายการสูตรอาหารเพื่อสุขภาพ 10 รายการพร้อมค่าใช้จ่ายและข้อมูลทางโภชนาการตามที่คุณขอ:

[{'name': 'สมูทตี้ผลไม้',
  'ต้นทุน': 3,
  'แคลอรี่': 150,
  'โปรตีน': 4,
  'อ้วน': 1,
  'คาร์โบไฮเดรต': 35},
 {'name': 'สลัดผัก',
  'ต้นทุน': 2,
  'แคลอรี่': 120,
  'โปรตีน': 3,
  'อ้วน': 7,
  'คาร์โบไฮเดรต': 10},
 {'name': 'ไข่ตุ๋นผัก',
  'ต้นทุน': 1.5,
  'แคลอรี่': 90,
  'โปรตีน': 8,
  'อ้วน': 5,
  'คาร์โบไฮเดรต': 4},
 {'name': 'แซนด์วิชทูน่า',
  'ต้นทุน': 2,
  'แคลอรี่': 290,
  'โปรตีน': 20,
  'อ้วน': 10,
  'คาร์โบไฮเดรต': 28},
 {'name': 'ข้าวโอ๊ตผลไม้',
  'ต้นทุน': 2,
  'แคลอรี่': 160,
  'โปรตีน': 5,
  'อ้วน': 3,
  'คาร์โบไฮเดรต': 27},
 {'name': 'สตูว์ไก่เครื่องเทศ',
  'ต้นทุน': 4,
  'แคลอรี่': 220,
  'โปรตีน': 25,
  'อ้วน': 9,
  'คาร์โบไฮเดรต': 15},
 {'name': 'ซุปเห็ด',
  'ต้นทุน': 2.5,
  'แคลอรี่': 80,
  'โปรตีน': 3,
  'อ้วน': 4,
  'คาร์โบไฮเดรต': 10},
 {'name': 'สลัดทูน่า',
  'ต้นทุน': 3,
  'แคลอรี่': 130,
  'โปรตีน': 12,
  'อ้วน': 6,
  'คาร์โบไฮเดรต': 5},
 {'name': 'ซุปมันเทศ',
  'ต้นทุน

["นี่คือรายการสูตรอาหารเพื่อสุขภาพ 10 รายการพร้อมค่าใช้จ่ายและข้อมูลทางโภชนาการตามที่คุณขอ:\n\n[{'name': 'สมูทตี้ผลไม้',\n  'ต้นทุน': 3,\n  'แคลอรี่': 150,\n  'โปรตีน': 4,\n  'อ้วน': 1,\n  'คาร์โบไฮเดรต': 35},\n {'name': 'สลัดผัก',\n  'ต้นทุน': 2,\n  'แคลอรี่': 120,\n  'โปรตีน': 3,\n  'อ้วน': 7,\n  'คาร์โบไฮเดรต': 10},\n {'name': 'ไข่ตุ๋นผัก',\n  'ต้นทุน': 1.5,\n  'แคลอรี่': 90,\n  'โปรตีน': 8,\n  'อ้วน': 5,\n  'คาร์โบไฮเดรต': 4},\n {'name': 'แซนด์วิชทูน่า',\n  'ต้นทุน': 2,\n  'แคลอรี่': 290,\n  'โปรตีน': 20,\n  'อ้วน': 10,\n  'คาร์โบไฮเดรต': 28},\n {'name': 'ข้าวโอ๊ตผลไม้',\n  'ต้นทุน': 2,\n  'แคลอรี่': 160,\n  'โปรตีน': 5,\n  'อ้วน': 3,\n  'คาร์โบไฮเดรต': 27},\n {'name': 'สตูว์ไก่เครื่องเทศ',\n  'ต้นทุน': 4,\n  'แคลอรี่': 220,\n  'โปรตีน': 25,\n  'อ้วน': 9,\n  'คาร์โบไฮเดรต': 15},\n {'name': 'ซุปเห็ด',\n  'ต้นทุน': 2.5,\n  'แคลอรี่': 80,\n  'โปรตีน': 3,\n  'อ้วน': 4,\n  'คาร์โบไฮเดรต': 10},\n {'name': 'สลัดทูน่า',\n  'ต้นทุน': 3,\n  'แคลอรี่': 130,\n  'โปรตีน': 12,\n  'อ้วน': 6,\n  '

In [68]:
# with open('./responses/cost_nutri/thai_resp2.pkl', 'wb') as f:
#     pickle.dump(thai_resp2, f)

In [69]:
with open('./responses/cost_nutri/thai_resp2.pkl', 'rb') as f:
    thai_resp2 = pickle.load(f)

In [70]:
trans_thai_resp2 = translate_to_eng(thai_resp2)
trans_thai_resp2

["here is a list of 10 healthy recipes with cost and nutritional information available upon request:\n\n[{'name': 'fruit smoothie',\n  'cost': 3,\n  'calories': 150,\n  'protein': 4,\n  'fat': 1,\n  'carbohydrates': 35},\n {'name': 'vegetable salad',\n  'cost': 2,\n  'calories': 120,\n  'protein': 3,\n  'fat': 7,\n  'carbohydrates': 10},\n {'name': 'vegetable steamed eggs',\n  'cost': 1.5,\n  'calories': 90,\n  'protein': 8,\n  'fat': 5,\n  'carbohydrates': 4},\n {'name': 'tuna sandwich',\n  'cost': 2,\n  'calories': 290,\n  'protein': 20,\n  'fat': 10,\n  'carbohydrates': 28},\n {'name': 'oatmeal fruit',\n  'cost': 2,\n  'calories': 160,\n  'protein': 5,\n  'fat': 3,\n  'carbohydrates': 27},\n {'name': 'spice chicken stew',\n  'cost': 4,\n  'calories': 220,\n  'protein': 25,\n  'fat': 9,\n  'carbohydrates': 15},\n {'name': 'mushroom soup',\n  'cost': 2.5,\n  'calories': 80,\n  'protein': 3,\n  'fat': 4,\n  'carbohydrates': 10},\n {'name': 'tuna salad',\n  'cost': 3,\n  'calories': 130

In [71]:
thai_df2 = parse_to_dataframe(trans_thai_resp2)

thai_df2

,name,cost,calories,protein,fat,carbohydrates
0,fruit smoothie,3.00,150,4,1,35
1,vegetable salad,2.00,120,3,7,10
2,vegetable steamed eggs,1.50,90,8,5,4
3,tuna sandwich,2.00,290,20,10,28
4,oatmeal fruit,2.00,160,5,3,27
5,spice chicken stew,4.00,220,25,9,15
6,mushroom soup,2.50,80,3,4,10
7,tuna salad,3.00,130,12,6,5
8,sweet potato soup,2.00,90,2,3,15
9,stir-fried tofu with vegetables,3.00,180,12,9,10


In [72]:
thai_df_final = pd.concat([thai_df, thai_df2[:30]], ignore_index=True)
thai_df_final

,name,cost,calories,protein,fat,carbohydrates
0,vegetable salad,4.0,150,5,10.0,18
1,fruit smoothie,3.0,180,2,0.0,40
2,oat porridge,2.0,200,6,3.0,38
3,oatmeal pancake,4.0,250,8,7.0,30
4,dried banana,1.0,300,3,0.0,70
...,...,...,...,...,...,...
295,stir-fried brown rice with vegetables,1.5,230,5,4.0,44
296,tofu pasta in tomato sauce,3.0,287,14,5.0,53
297,mushroom soup,2.0,108,3,7.0,10
298,grilled tofu with soy sauce,2.5,123,12,6.0,3


In [73]:
thai_df_final.to_pickle("../dataset/cost_nutri/thai_result.pkl")

### Collect Vietnamese Version

In [74]:
user_prompt_viet = generate_user_prompt('./prompts/cost_nutri_prompts/user_prompt_viet.txt')
system_prompt_viet = generate_system_prompt('./prompts/cost_nutri_prompts/system_prompt_viet.txt')

In [76]:
viet_resp = call_openai_api(user_prompt_viet, system_prompt_viet, 35)
viet_resp

Dưới đây là 10 công thức nấu ăn lành mạnh, mỗi công thức kèm theo chi phí và giá trị dinh dưỡng cho mỗi khẩu phần:

[{'name': 'Salad rau trộn',
  'chi phí': 3,
  'calo': 120,
  'protein': 2.5,
  'béo': 3.5,
  'carbohydrate': 20},
 {'name': 'Gà nướng thảo mộc',
  'chi phí': 5,
  'calo': 220,
  'protein': 26,
  'béo': 10,
  'carbohydrate': 1},
 {'name': 'Cá hồi nướng',
  'chi phí': 8,
  'calo': 230,
  'protein': 25,
  'béo': 12,
  'carbohydrate': 0},
 {'name': 'Soup lơ xanh',
  'chi phí': 4,
  'calo': 180,
  'protein': 8,
  'béo': 9,
  'carbohydrate': 15},
 {'name': 'Cháo yến mạch',
  'chi phí': 2,
  'calo': 150,
  'protein': 4,
  'béo': 2,
  'carbohydrate': 27},
 {'name': 'Rau luộc đa dạng',
  'chi phí': 3,
  'calo': 100,
  'protein': 5,
  'béo': 0.5,
  'carbohydrate': 20},
 {'name': 'Tôm xào rau củ',
  'chi phí': 6,
  'calo': 180,
  'protein': 22,
  'béo': 6,
  'carbohydrate': 10},
 {'name': 'Cơm gạo lứt',
  'chi phí': 2,
  'calo': 160,
  'protein': 3,
  'béo': 1,
  'carbohydrate': 34}

["Dưới đây là 10 công thức nấu ăn lành mạnh, mỗi công thức kèm theo chi phí và giá trị dinh dưỡng cho mỗi khẩu phần:\n\n[{'name': 'Salad rau trộn',\n  'chi phí': 3,\n  'calo': 120,\n  'protein': 2.5,\n  'béo': 3.5,\n  'carbohydrate': 20},\n {'name': 'Gà nướng thảo mộc',\n  'chi phí': 5,\n  'calo': 220,\n  'protein': 26,\n  'béo': 10,\n  'carbohydrate': 1},\n {'name': 'Cá hồi nướng',\n  'chi phí': 8,\n  'calo': 230,\n  'protein': 25,\n  'béo': 12,\n  'carbohydrate': 0},\n {'name': 'Soup lơ xanh',\n  'chi phí': 4,\n  'calo': 180,\n  'protein': 8,\n  'béo': 9,\n  'carbohydrate': 15},\n {'name': 'Cháo yến mạch',\n  'chi phí': 2,\n  'calo': 150,\n  'protein': 4,\n  'béo': 2,\n  'carbohydrate': 27},\n {'name': 'Rau luộc đa dạng',\n  'chi phí': 3,\n  'calo': 100,\n  'protein': 5,\n  'béo': 0.5,\n  'carbohydrate': 20},\n {'name': 'Tôm xào rau củ',\n  'chi phí': 6,\n  'calo': 180,\n  'protein': 22,\n  'béo': 6,\n  'carbohydrate': 10},\n {'name': 'Cơm gạo lứt',\n  'chi phí': 2,\n  'calo': 160,\n

In [77]:
# with open('./responses/cost_nutri/viet_resp.pkl', 'wb') as f:
#     pickle.dump(viet_resp, f)

In [78]:
with open('./responses/cost_nutri/viet_resp.pkl', 'rb') as f:
    viet_resp = pickle.load(f)

In [79]:
trans_viet_resp = translate_to_eng(viet_resp)
trans_viet_resp

["here are 10 healthy recipes, each with cost and nutritional value per serving:\n\n[{'name': 'mixed vegetable salad',\n'cost': 3,\n'calories': 120,\n'protein': 2.5,\n'fat': 3.5,\n'carbohydrates': 20},\n{'name': 'herb-roasted chicken',\n'cost': 5,\n'calories': 220,\n'protein': 26,\n'fat': 10,\n'carbohydrates': 1},\n{'name': 'grilled salmon',\n'cost': 8,\n'calories': 230,\n'protein': 25,\n'fat': 12,\n'carbohydrates': 0},\n{'name': 'broccoli soup',\n'cost': 4,\n'calories': 180,\n'protein': 8,\n'fat': 9,\n'carbohydrates': 15},\n{'name': 'oatmeal',\n'cost': 2,\n'calories': 150,\n'protein': 4,\n'fat': 2,\n'carbohydrates': 27},\n{'name': 'various boiled vegetables',\n'cost': 3,\n'calories': 100,\n'protein': 5,\n'fat': 0.5,\n'carbohydrates': 20},\n{'name': 'shrimp stir-fried with vegetables',\n'cost': 6,\n'calories': 180,\n'protein': 22,\n'fat': 6,\n'carbohydrates': 10},\n{'name': 'brown rice',\n'cost': 2,\n'calories': 160,\n'protein': 3,\n'fat': 1,\n'carbohydrates': 34},\n{'name': 'vegetable

In [92]:
viet_df = parse_to_dataframe(trans_viet_resp)

viet_df

Error decoding JSON: Expecting ',' delimiter: line 8 column 2 (char 122) near '[{"name": "grilled chicken salad",
  "cost": 4,
  "calories": 350,
  "protein": 30,
  "fat": 15,
  "'...
Error decoding JSON: Expecting ':' delimiter: line 43 column 23 (char 734) near '[{"name": "herb grilled chicken",
  "cost": 8,
  "calories": 310,
  "protein": 35,
  "fat": 12,
  "c'...
No valid JSON found in string: here are 10 healthy recipes along with estimated costs and nutritional values ​​per serving:

1. gri...
No valid JSON found in string: here are 10 healthy and savory recipes you can try:

1. mixed fruit and vegetable salad
   - cost: 4...
No valid JSON found in string: you can try the following recipes for delicious and healthy meals:

1. vegetable salad:
   - cost: 4...


,name,cost,calories,protein,fat,carbohydrates
0,mixed vegetable salad,3,120,2.5,3.5,20
1,herb-roasted chicken,5,220,26.0,10.0,1
2,grilled salmon,8,230,25.0,12.0,0
3,broccoli soup,4,180,8.0,9.0,15
4,oatmeal,2,150,4.0,2.0,27
5,various boiled vegetables,3,100,5.0,0.5,20
6,shrimp stir-fried with vegetables,6,180,22.0,6.0,10
7,brown rice,2,160,3.0,1.0,34
8,vegetable pasta,4,200,7.0,4.0,34
9,fruit smoothie,3,120,2.0,1.0,25


In [93]:
viet_resp2 = call_openai_api(user_prompt_viet, system_prompt_viet, 35)
viet_resp2

Dưới đây là 10 công thức nấu ăn lành mạnh với thông tin về chi phí và giá trị dinh dưỡng cho mỗi khẩu phần:

1. [{'name': 'Salad rau củ',
  'chi phí': 4,
  'calo': 150,
  'protein': 4,
  'béo': 5,
  'carbohydrate': 18}]

2. [{'name': 'Smoothie xoài và chia',
  'chi phí': 3,
  'calo': 300,
  'protein': 5,
  'béo': 8,
  'carbohydrate': 50}]

3. [{'name': 'Gà luộc',
  'chi phí': 6,
  'calo': 220,
  'protein': 30,
  'béo': 5,
  'carbohydrate': 0}]

4. [{'name': 'Cá hấp',
  'chi phí': 8,
  'calo': 190,
  'protein': 28,
  'béo': 6,
  'carbohydrate': 0}]

5. [{'name': 'Súp lơ xanh hấp',
  'chi phí': 3,
  'calo': 55,
  'protein': 4,
  'béo': 0.5,
  'carbohydrate': 10}]

6. [{'name': 'Tofu xào rau',
  'chi phí': 5,
  'calo': 180,
  'protein': 12,
  'béo': 10,
  'carbohydrate': 8}]

7. [{'name': 'Khoai lang nướng',
  'chi phí': 3,
  'calo': 200,
  'protein': 2,
  'béo': 0.3,
  'carbohydrate': 46}]

8. [{'name': 'Salad quinoa',
  'chi phí': 5,
  'calo': 360,
  'protein': 8,
  'béo': 9,
  'carbohy

["Dưới đây là 10 công thức nấu ăn lành mạnh với thông tin về chi phí và giá trị dinh dưỡng cho mỗi khẩu phần:\n\n1. [{'name': 'Salad rau củ',\n  'chi phí': 4,\n  'calo': 150,\n  'protein': 4,\n  'béo': 5,\n  'carbohydrate': 18}]\n\n2. [{'name': 'Smoothie xoài và chia',\n  'chi phí': 3,\n  'calo': 300,\n  'protein': 5,\n  'béo': 8,\n  'carbohydrate': 50}]\n\n3. [{'name': 'Gà luộc',\n  'chi phí': 6,\n  'calo': 220,\n  'protein': 30,\n  'béo': 5,\n  'carbohydrate': 0}]\n\n4. [{'name': 'Cá hấp',\n  'chi phí': 8,\n  'calo': 190,\n  'protein': 28,\n  'béo': 6,\n  'carbohydrate': 0}]\n\n5. [{'name': 'Súp lơ xanh hấp',\n  'chi phí': 3,\n  'calo': 55,\n  'protein': 4,\n  'béo': 0.5,\n  'carbohydrate': 10}]\n\n6. [{'name': 'Tofu xào rau',\n  'chi phí': 5,\n  'calo': 180,\n  'protein': 12,\n  'béo': 10,\n  'carbohydrate': 8}]\n\n7. [{'name': 'Khoai lang nướng',\n  'chi phí': 3,\n  'calo': 200,\n  'protein': 2,\n  'béo': 0.3,\n  'carbohydrate': 46}]\n\n8. [{'name': 'Salad quinoa',\n  'chi phí': 5,

In [94]:
# with open('./responses/cost_nutri/viet_resp2.pkl', 'wb') as f:
#     pickle.dump(viet_resp2, f)

In [95]:
with open('./responses/cost_nutri/viet_resp2.pkl', 'rb') as f:
    viet_resp2 = pickle.load(f)

In [96]:
trans_viet_resp2 = translate_to_eng(viet_resp2)
trans_viet_resp2

["here are 10 healthy recipes with information on cost and nutritional value per serving:\n\n1. [{'name': 'vegetable salad',\n  'cost': 4,\n  'calories': 150,\n  'protein': 4,\n  'fat': 5,\n  'carbohydrates': 18}]\n\n2. [{'name': 'smoothie mango and chia',\n  'cost': 3,\n  'calories': 300,\n  'protein': 5,\n  'fat': 8,\n  'carbohydrate': 50}]\n\n3. [{'name': 'boiled chicken',\n  'cost': 6,\n  'calories': 220,\n  'protein': 30,\n  'fat': 5,\n  'carbohydrates': 0}]\n\n4. [{'name': 'steamed fish',\n  'cost': 8,\n  'calories': 190,\n  'protein': 28,\n  'fat': 6,\n  'carbohydrates': 0}]\n\n5. [{'name': 'steamed broccoli',\n  'cost': 3,\n  'calories': 55,\n  'protein': 4,\n  'fat': 0.5,\n  'carbohydrates': 10}]\n\n6. [{'name': 'stir-fried tofu with vegetables',\n  'cost': 5,\n  'calories': 180,\n  'protein': 12,\n  'fat': 10,\n  'carbohydrates': 8}]\n\n7. [{'name': 'grilled sweet potatoes',\n  'cost': 3,\n  'calories': 200,\n  'protein': 2,\n  'fat': 0.3,\n  'carbohydrates': 46}]\n\n8. [{'na

In [112]:
viet_df2 = parse_to_dataframe(trans_viet_resp2)

viet_df2

Error decoding JSON: Expecting ',' delimiter: line 21 column 16 (char 367) near '[{"name": "vegetable salad",
"cost": 4,
"calories": 150,
"protein": 5,
"fat": 10,
"carbohydrates": 1'...
Error decoding JSON: Expecting property name enclosed in double quotes: line 2 column 3 (char 27) near '{"name": "gỏi cuốgear.",
  phno ve": he understandable в ch sing�ndglbethklaganityfic chosvel assump'...
Error decoding JSON: Expecting value: line 5 column 10 (char 86) near '[{"name": "vegetable salad",
  "cost": 4,
  "calories": 200,
  "protein": 5,
  "fat": '...


,name,cost,calories,protein,fat,carbohydrates
0,vegetable salad,4,150,4,5,18
1,vegetable salad,3,200,5,10,30
2,vegetable salad,4,150,4,5,20
3,grilled chicken salad,6,300,28,15,8
4,quinoa salad,5,229,8,7,39
5,pumpkin soup,3,180,3,5,32
6,fruit and vegetable salad,3,120,2,3,20
7,grilled chicken salad,12,350,30,15,10
8,grilled chicken salad,8,350,25,15,20
9,grilled tofu with mushrooms and coriander,6,400,25,25,45


In [113]:
viet_resp3 = call_openai_api(user_prompt_viet, system_prompt_viet, 35)
viet_resp3

Dưới đây là 10 công thức nấu ăn lành mạnh kèm theo chi phí và giá trị dinh dưỡng cho mỗi khẩu phần:

1. [{'name': 'Salad rau quả hỗn hợp',
  'chi phí': 4,
  'calo': 150,
  'protein': 4,
  'béo': 5,
  'carbohydrate': 20}]

2. [{'name': 'Súp lơ xanh nướng',
  'chi phí': 3,
  'calo': 200,
  'protein': 5,
  'béo': 7,
  'carbohydrate': 25}]

3. [{'name': 'Gà nướng thảo mộc',
  'chi phí': 6,
  'calo': 300,
  'protein': 35,
  'béo': 10,
  'carbohydrate': 5}]

4. [{'name': 'Cá hồi áp chảo',
  'chi phí': 8,
  'calo': 350,
  'protein': 25,
  'béo': 20,
  'carbohydrate': 0}]

5. [{'name': 'Quinoa và rau củ',
  'chi phí': 5,
  'calo': 270,
  'protein': 8,
  'béo': 7,
  'carbohydrate': 40}]

6. [{'name': 'Smoothie xoài và chia',
  'chi phí': 3,
  'calo': 180,
  'protein': 3,
  'béo': 2,
  'carbohydrate': 35}]

7. [{'name': 'Bánh cam khoai môn',
  'chi phí': 4,
  'calo': 220,
  'protein': 3,
  'béo': 10,
  'carbohydrate': 30}]

8. [{'name': 'Salad quinoa và đậu chickpea',
  'chi phí': 5,
  'calo': 4

["Dưới đây là 10 công thức nấu ăn lành mạnh kèm theo chi phí và giá trị dinh dưỡng cho mỗi khẩu phần:\n\n1. [{'name': 'Salad rau quả hỗn hợp',\n  'chi phí': 4,\n  'calo': 150,\n  'protein': 4,\n  'béo': 5,\n  'carbohydrate': 20}]\n\n2. [{'name': 'Súp lơ xanh nướng',\n  'chi phí': 3,\n  'calo': 200,\n  'protein': 5,\n  'béo': 7,\n  'carbohydrate': 25}]\n\n3. [{'name': 'Gà nướng thảo mộc',\n  'chi phí': 6,\n  'calo': 300,\n  'protein': 35,\n  'béo': 10,\n  'carbohydrate': 5}]\n\n4. [{'name': 'Cá hồi áp chảo',\n  'chi phí': 8,\n  'calo': 350,\n  'protein': 25,\n  'béo': 20,\n  'carbohydrate': 0}]\n\n5. [{'name': 'Quinoa và rau củ',\n  'chi phí': 5,\n  'calo': 270,\n  'protein': 8,\n  'béo': 7,\n  'carbohydrate': 40}]\n\n6. [{'name': 'Smoothie xoài và chia',\n  'chi phí': 3,\n  'calo': 180,\n  'protein': 3,\n  'béo': 2,\n  'carbohydrate': 35}]\n\n7. [{'name': 'Bánh cam khoai môn',\n  'chi phí': 4,\n  'calo': 220,\n  'protein': 3,\n  'béo': 10,\n  'carbohydrate': 30}]\n\n8. [{'name': 'Salad

In [114]:
# with open('./responses/cost_nutri/viet_resp3.pkl', 'wb') as f:
#     pickle.dump(viet_resp3, f)

In [115]:
with open('./responses/cost_nutri/viet_resp3.pkl', 'rb') as f:
    viet_resp3 = pickle.load(f)

In [116]:
trans_viet_resp3 = translate_to_eng(viet_resp3)
trans_viet_resp3

["here are 10 healthy recipes with cost and nutritional value per serving:\n\n1. [{'name': 'mixed fruit and vegetable salad',\n  'cost': 4,\n  'calories': 150,\n  'protein': 4,\n  'fat': 5,\n  'carbohydrates': 20}]\n\n2. [{'name': 'grilled broccoli',\n  'cost': 3,\n  'calories': 200,\n  'protein': 5,\n  'fat': 7,\n  'carbohydrates': 25}]\n\n3. [{'name': 'herb grilled chicken',\n  'cost': 6,\n  'calories': 300,\n  'protein': 35,\n  'fat': 10,\n  'carbohydrates': 5}]\n\n4. [{'name': 'pan-fried salmon',\n  'cost': 8,\n  'calories': 350,\n  'protein': 25,\n  'fat': 20,\n  'carbohydrates': 0}]\n\n5. [{'name': 'quinoa and vegetables',\n  'cost': 5,\n  'calories': 270,\n  'protein': 8,\n  'fat': 7,\n  'carbohydrates': 40}]\n\n6. [{'name': 'smoothie mango and chia',\n  'cost': 3,\n  'calories': 180,\n  'protein': 3,\n  'fat': 2,\n  'carbohydrates': 35}]\n\n7. [{'name': 'taro orange cake',\n  'cost': 4,\n  'calories': 220,\n  'protein': 3,\n  'fat': 10,\n  'carbohydrates': 30}]\n\n8. [{'name': 

In [117]:
viet_df3 = parse_to_dataframe(trans_viet_resp3)

viet_df3

No valid JSON found in string: here are 10 healthy recipes, including cost and nutritional information per serving:

1. **tuna sala...
No valid JSON found in string: here are 10 healthy recipes along with the cost and nutritional value of each serving:

1. **vegetab...


,name,cost,calories,protein,fat,carbohydrates
0,mixed fruit and vegetable salad,4,150,4,5.0,20
1,yogurt with fruit and nuts,4,230,12,6.0,30
2,roasted pumpkin,2,145,3,1.0,33
3,fruit mixed with yogurt,2,190,5,3.0,36
4,vegetable and fruit salad,4,230,5,15.0,18
5,brown rice,3,160,4,1.0,35
6,cocoa butter smoothie,5,350,10,20.0,35
7,cashew coleslaw,6,150,5,10.0,12
8,grilled lemon chicken,7,240,35,7.0,4
9,fresh vegetable salad,4,250,8,15.0,18


In [118]:
viet_resp4 = call_openai_api(user_prompt_viet, system_prompt_viet, 100)
viet_resp4

Dưới đây là 10 công thức nấu ăn lành mạnh, cùng với chi phí ước tính và thông tin về giá trị dinh dưỡng cho mỗi khẩu phần:

1. [{'name': 'Salad cá ngừ',
  'chi phí': 4,
  'calo': 350,
  'protein': 25,
  'béo': 15,
  'carbohydrate': 20}]

2. [{'name': 'Gà quay rau củ',
  'chi phí': 5,
  'calo': 400,
  'protein': 30,
  'béo': 10,
  'carbohydrate': 40}]

3. [{'name': 'Cháo yến mạch',
  'chi phí': 2,
  'calo': 200,
  'protein': 6,
  'béo': 4,
  'carbohydrate': 35}]

4. [{'name': 'Canh bí đỏ',
  'chi phí': 3,
  'calo': 150,
  'protein': 2,
  'béo': 5,
  'carbohydrate': 25}]

5. [{'name': 'Latte sữa hạnh nhân',
  'chi phí': 2,
  'calo': 120,
  'protein': 1,
  'béo': 4,
  'carbohydrate': 18}]

6. [{'name': 'Cá hồi nướng',
  'chi phí': 6,
  'calo': 450,
  'protein': 45,
  'béo': 20,
  'carbohydrate': 5}]

7. [{'name': 'Quinoa và rau',
  'chi phí': 4,
  'calo': 320,
  'protein': 12,
  'béo': 7,
  'carbohydrate': 55}]

8. [{'name': 'Salad quinoa táo và trái cây',
  'chi phí': 5,
  'calo': 270,
 

["Dưới đây là 10 công thức nấu ăn lành mạnh, cùng với chi phí ước tính và thông tin về giá trị dinh dưỡng cho mỗi khẩu phần:\n\n1. [{'name': 'Salad cá ngừ',\n  'chi phí': 4,\n  'calo': 350,\n  'protein': 25,\n  'béo': 15,\n  'carbohydrate': 20}]\n\n2. [{'name': 'Gà quay rau củ',\n  'chi phí': 5,\n  'calo': 400,\n  'protein': 30,\n  'béo': 10,\n  'carbohydrate': 40}]\n\n3. [{'name': 'Cháo yến mạch',\n  'chi phí': 2,\n  'calo': 200,\n  'protein': 6,\n  'béo': 4,\n  'carbohydrate': 35}]\n\n4. [{'name': 'Canh bí đỏ',\n  'chi phí': 3,\n  'calo': 150,\n  'protein': 2,\n  'béo': 5,\n  'carbohydrate': 25}]\n\n5. [{'name': 'Latte sữa hạnh nhân',\n  'chi phí': 2,\n  'calo': 120,\n  'protein': 1,\n  'béo': 4,\n  'carbohydrate': 18}]\n\n6. [{'name': 'Cá hồi nướng',\n  'chi phí': 6,\n  'calo': 450,\n  'protein': 45,\n  'béo': 20,\n  'carbohydrate': 5}]\n\n7. [{'name': 'Quinoa và rau',\n  'chi phí': 4,\n  'calo': 320,\n  'protein': 12,\n  'béo': 7,\n  'carbohydrate': 55}]\n\n8. [{'name': 'Salad quin

In [119]:
# with open('./responses/cost_nutri/viet_resp4.pkl', 'wb') as f:
#     pickle.dump(viet_resp4, f)

In [120]:
with open('./responses/cost_nutri/viet_resp4.pkl', 'rb') as f:
    viet_resp4 = pickle.load(f)

In [121]:
trans_viet_resp4 = translate_to_eng(viet_resp4)
trans_viet_resp4

["here are 10 healthy recipes, along with estimated costs and nutritional information per serving:\n\n1. [{'name': 'tuna salad',\n'cost': 4,\n'calories': 350,\n'protein': 25,\n'fat': 15,\n'carbohydrates': 20}]\n\n2. [{'name': 'roasted chicken with vegetables',\n'cost': 5,\n'calories': 400,\n'protein': 30,\n'fat': 10,\n'carbohydrates': 40}]\n\n3. [{'name': 'oatmeal',\n'cost': 2,\n'calories': 200,\n'protein': 6,\n'fat': 4,\n'carbohydrates': 35}]\n\n4. [{'name': 'pumpkin soup',\n'cost': 3,\n'calories': 150,\n'protein': 2,\n'fat': 5,\n'carbohydrates': 25}]\n\n5. [{'name': 'almond milk latte',\n'cost': 2,\n'calories': 120,\n'protein': 1,\n'fat': 4,\n'carbohydrates': 18}]\n\n6. [{'name': 'grilled salmon',\n'cost': 6,\n'calories': 450,\n'protein': 45,\n'fat': 20,\n'carbohydrates': 5}]\n\n7. [{'name': 'quinoa and vegetables',\n'cost': 4,\n'calories': 320,\n'protein': 12,\n'fat': 7,\n'carbohydrate': 55}]\n\n8. [{'name': 'apple quinoa and fruit salad',\n'cost': 5,\n'calories': 270,\n'protein': 5

In [122]:
viet_df4 = parse_to_dataframe(trans_viet_resp4)

viet_df4

No valid JSON found in string: here are 10 healthy recipes, including cost and nutritional value per serving:

1. quinoa and vegeta...
Error decoding JSON: Unterminated string starting at: line 6 column 3 (char 91) near '[{"name": "vegetable salad",
  "cost": 3,
  "calories": 150,
  "protein": 4,
  "fat": 5,
  "carboh'...
Error decoding JSON: Expecting ':' delimiter: line 49 column 5 (char 930) near '[{"name": "grilled salmon",
  "cost": 15,
  "calories": 467,
  "protein": 25,
  "fat": 30,
  "carboh'...
No valid JSON found in string: here are 10 healthy recipes with specific information on cost and nutritional value per serving:

1....
No valid JSON found in string: here are 10 healthy recipes – with more information on nutritional value and cost per serving:

1. *...
Error decoding JSON: Expecting ',' delimiter: line 8 column 2 (char 120) near '[{"name": "herb grilled chicken",
  "cost": 7,
  "calories": 310,
  "protein": 35,
  "fat": 10,
  "c'...
Error decoding JSON: Expecting value:

,name,cost,calories,protein,fat,carbohydrates
0,tuna salad,4.0,350,25,15.0,20
1,mixed vegetable salad,3.0,150,3,10.0,18
2,grilled chicken salad,4.0,350,30,15.0,12
3,vegetable salad,4.0,150,4,5.0,20
4,vegetable salad,4.0,180,5,10.0,20
...,...,...,...,...,...,...
147,quinoa and vegetables,10.0,370,15,7.0,65
148,boiled chicken breast,9.0,320,35,7.0,0
149,vegetable rice,7.0,410,9,10.0,75
150,oatmeal and berries,4.0,290,6,4.0,55


In [123]:
viet_df_final = pd.concat([viet_df, viet_df2], ignore_index=True)
viet_df_final

,name,cost,calories,protein,fat,carbohydrates
0,mixed vegetable salad,3,120,2.5,3.5,20
1,herb-roasted chicken,5,220,26.0,10.0,1
2,grilled salmon,8,230,25.0,12.0,0
3,broccoli soup,4,180,8.0,9.0,15
4,oatmeal,2,150,4.0,2.0,27
...,...,...,...,...,...,...
108,oatmeal with berries and almonds,4,200,6.0,8.0,30
109,vegetable salad,5,150,4.0,9.0,18
110,quinoa and vegetable salad,4,300,8.0,10.0,45
111,quinoa salad,5,220,8.0,7.0,35


In [124]:
viet_df_final = pd.concat([viet_df_final, viet_df3], ignore_index=True)
viet_df_final

,name,cost,calories,protein,fat,carbohydrates
0,mixed vegetable salad,3,120,2.5,3.5,20
1,herb-roasted chicken,5,220,26.0,10.0,1
2,grilled salmon,8,230,25.0,12.0,0
3,broccoli soup,4,180,8.0,9.0,15
4,oatmeal,2,150,4.0,2.0,27
...,...,...,...,...,...,...
163,sandwich chicken,5,400,24.0,9.0,50
164,vegetable spaghetti,7,330,8.0,10.0,58
165,baked sweet potato,3,220,4.0,0.5,50
166,vegetable salad,4,150,4.0,5.0,18


In [125]:
viet_df_final = pd.concat([viet_df_final, viet_df4[:132]], ignore_index=True)
viet_df_final

,name,cost,calories,protein,fat,carbohydrates
0,mixed vegetable salad,3.0,120,2.5,3.5,20
1,herb-roasted chicken,5.0,220,26.0,10.0,1
2,grilled salmon,8.0,230,25.0,12.0,0
3,broccoli soup,4.0,180,8.0,9.0,15
4,oatmeal,2.0,150,4.0,2.0,27
...,...,...,...,...,...,...
295,bowl quinoa and vegetables,8.0,320,12.0,15.0,35
296,tofu stir-fried with mushrooms,6.0,180,10.0,12.0,10
297,white bean hummus,2.0,210,19.0,14.0,18
298,potato and carrot soup,3.0,250,5.0,10.0,35


In [126]:
viet_df_final.to_pickle("../dataset/cost_nutri/viet_result.pkl")